In [2]:
import os
import csv
import datetime

file_path = 'C:/Users/Sowmya/QOE_CWI/Selected_PID_data/P14/P14_ECG_S1.csv'  # Replace with your file path
timestamp_format = '%Y/%m/%d %H:%M:%S.%f'
            
with open(file_path, 'r') as file:
    csv_reader = csv.reader(file, delimiter=',')  # Assuming tab-separated values
    headers = next(csv_reader)  # Read the headers
    #second_row_headers = next(csv_reader)  # Read the second row of headers
    
    data = []
    for row in csv_reader:
        timestamp_str = row[1]  # Assuming timestamp is in the first column
        #timestamp_str1= timestamp_str.split(',')[1] 
        timestamp = datetime.datetime.strptime(timestamp_str, timestamp_format)
        row[1] = timestamp
        data.append(row)

import csv
import datetime

# Function to find the nearest timestamp in the CSV file
def find_nearest_timestamp(data1, target_timestamp):
    nearest_timestamp = None
    min_difference = float('inf')

    for row in data1:
            # Assuming the timestamp is in the first column of each row
            #csv_timestamp = datetime.datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S.%f')  # Adjust the datetime format as per your CSV
            csv_timestamp = row[1]
            # Calculate the difference in timestamps
            difference = abs((csv_timestamp - target_timestamp).total_seconds())  # Get the absolute difference in seconds

            if difference < min_difference:
                min_difference = difference
                nearest_timestamp = csv_timestamp
                index = row
    return nearest_timestamp, index
            
directory = 'C:/Users/Sowmya/QOE_CWI/Selected_PID_data/P14/RCQoEA data/'

# List all files in the directory
files = os.listdir(directory)

# Iterate through the files in the directory
for file_name in files:
    if file_name.endswith('.txt') and file_name.startswith('PostQuestion_'):
        # Split the filename by underscores and remove the file extension
        file_parts = file_name.replace('.txt', '').split('_')

        # Extract the timestamp and other details
        participant_id = file_parts[1]
        video_id = file_parts[2]
        date_time= file_parts[3]  # Includes the date and time together

        # Extract the date and time from the timestamp
        year = date_time[:4]
        month = date_time[4:6]
        date = date_time[6:8]
        hour = date_time[9:11]  # Removing 'T' from the timestamp
        minute = date_time[11:13]
        second = date_time[13:15]
        # Create the full file path
        file_path = os.path.join(directory, file_name)

        # Open and read the file contents
        with open(file_path, 'r') as file:
            content = file.read()
            # Process the content as needed
            # Example: Print the content of the file along with the extracted details
            print(f"\nFile: {file_name}")
            
            # Splitting the content by newline character to get individual lines
            lines = content.split('\n')

            # Initializing lists to store StartTime and StopTime data
            start_times = []
            stop_times = []

            # Processing each line
            for line in lines:
            # Splitting each line by semicolon to extract individual values
                values = line.split(';')
    
                # Check if the line contains StartTime or StopTime data
                if len(values) > 1:
                    if values[1].startswith("StartTime"):
                        start_times.append(values[1].split(': ')[1])
                    elif values[1].startswith("StopTime"):
                        stop_times.append(values[1].split(': ')[1])
           
            # Convert start_times to datetime objects and append the hour
            for start_time in start_times:
                # Splitting the time components
                parts = start_time.split(':')
                # Extracting hour, minute, second, and millisecond
                minute1 = int(parts[0])
                second1 = int(parts[1])
                millisecond = int(parts[2])
    
                # Convert milliseconds to microseconds (1 millisecond = 1000 microseconds)
                microsecond = millisecond * 1000
                # Creating a datetime object by adding the hour, minute, second, and microsecond
                start_timestamp = datetime.datetime(int(year), int(month), int(date), int(hour), minute1, second1, microsecond)
                start_timestamp_without_ms = start_timestamp.replace(microsecond=0)
            #print('Start Timestamp:', start_timestamp)

           
            # Convert stop_times to datetime objects and append the hour
            for stop_time in stop_times:
                # Splitting the time components
                parts = stop_time.split(':')
                # Extracting hour, minute, second, and millisecond
                stop_minute = int(parts[0])
                if minute1 == 59 and stop_minute == 0:
                    stop_hour = int(hour)+1
                else:
                    stop_hour = int(hour)
                second2 = int(parts[1])
                millisecond2 = int(parts[2])
                    
                # Convert milliseconds to microseconds (1 millisecond = 1000 microseconds)
                microsecond2 = millisecond2 * 1000
                # Creating a datetime object by adding the hour, minute, second, and microsecond
                stop_timestamp = datetime.datetime(int(year), int(month), int(date), stop_hour, stop_minute, second2, microsecond2)
                stop_timestamp_without_ms = stop_timestamp.replace(microsecond=0)
            #print('Stop Timestamp:', stop_timestamp)
            
        # Find the nearest timestamps to start and stop times in the CSV file
        nearest_start_timestamp, start_index = find_nearest_timestamp(data, start_timestamp)
        nearest_stop_timestamp, stop_index = find_nearest_timestamp(data, stop_timestamp)

        #print(f"The nearest start timestamp in the CSV file is: {nearest_start_timestamp}")
        #print(f"The nearest stop timestamp in the CSV file is: {nearest_stop_timestamp}")
        #print(f"The index of nearest stop timestamp in the CSV file is: {stop_index}")
        
        # Initialize a list to store data between start and stop times
        filtered_data = []

        # Create the filename
        filename = f"C:/Users/Sowmya/QOE_CWI/Selected_PID_data/P14/RCQoEA data/ecg_P{participant_id}_{video_id}.csv"
              
        # Associate each data point with its corresponding timestamp
        for entry in data: # Get the timestamp from data
            data_timestamp = entry[1]
            data_value = entry[2].strip('[]')
            # Remove milliseconds from the data timestamp
            #data_timestamp_without_ms = data_timestamp.replace(microsecond=0)
            
            # Check if the timestamp is between the nearest start and stop timestamps
            if nearest_start_timestamp <= data_timestamp <= nearest_stop_timestamp:
                filtered_data.append([data_timestamp, data_value])

            #if start_timestamp_without_ms <= data_timestamp_without_ms  <= stop_timestamp_without_ms:
                #print("Data timestamp matches the stop timestamp (without milliseconds).")
            
        # Check if we have collected 3899 samples and add an extra sample to reach 3900
        if len(filtered_data) == 3899:
               
            # Check if the stop timestamp exists in the filtered data and add an extra sample if possible
            if stop_index != -1:
                # Get the next sample after the stop timestamp if available
                extra_sample = data[data.index(stop_index)+1]
                #print(f"Extra sample after the nearest stop timestamp: {extra_sample}")
                filtered_data.append([extra_sample[1],extra_sample[2].strip('[]')])  # Append the extra sample to reach 3900 samples
                
            else:
                print("Nearest stop timestamp not found in the filtered data.")
        else:
            print("Required number of samples (3899) has been reached.")
   

        # Write the filtered data to a CSV file
        
        with open(filename, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(headers[1:])  # Write header
            csv_writer.writerows(filtered_data)

        print(f"Filtered data between {start_timestamp} and {stop_timestamp} written to {filename}.")                
        
         


File: PostQuestion_14_10_20230810T095845.txt
Required number of samples (3899) has been reached.
Filtered data between 2023-08-10 09:58:45.921000 and 2023-08-10 09:59:15.897000 written to C:/Users/Sowmya/QOE_CWI/Selected_PID_data/P14/RCQoEA data/ecg_P14_10.csv.

File: PostQuestion_14_11_20230810T092531.txt
Filtered data between 2023-08-10 09:25:31.237000 and 2023-08-10 09:26:01.213000 written to C:/Users/Sowmya/QOE_CWI/Selected_PID_data/P14/RCQoEA data/ecg_P14_11.csv.

File: PostQuestion_14_12_20230810T092741.txt
Filtered data between 2023-08-10 09:27:41.510000 and 2023-08-10 09:28:11.486000 written to C:/Users/Sowmya/QOE_CWI/Selected_PID_data/P14/RCQoEA data/ecg_P14_12.csv.

File: PostQuestion_14_13_20230810T095723.txt
Required number of samples (3899) has been reached.
Filtered data between 2023-08-10 09:57:23.670000 and 2023-08-10 09:57:53.645000 written to C:/Users/Sowmya/QOE_CWI/Selected_PID_data/P14/RCQoEA data/ecg_P14_13.csv.

File: PostQuestion_14_14_20230810T092857.txt
Filter

In [21]:
extra_sample = data[data.index(stop_index)+1]
extra_sample[1]
extra_sample[2].strip('[]')

'-76.0'

In [5]:
file_path = 'C:/Users/Sowmya/QOE_CWI/Selected_PID_data/P47/P47_ECG.csv'  # Replace with your file path
timestamp_format = '%Y/%m/%d %H:%M:%S.%f'
            
with open(file_path, 'r') as file:
    csv_reader = csv.reader(file, delimiter=',')  # Assuming tab-separated values
    headers = next(csv_reader)  # Read the headers
    second_row_headers = next(csv_reader)  # Read the second row of headers
    
    data = []
    for row in csv_reader:
        timestamp_str = row[0]  # Assuming timestamp is in the first column
        #timestamp_str1= timestamp_str.split(',')[1] 
        timestamp = datetime.datetime.strptime(timestamp_str, timestamp_format)
        row[0] = timestamp
        data.append(row)
            
print("\nData:")
for row in data[:5]:
    print(row)


Data:
[datetime.datetime(2023, 12, 13, 12, 37, 19, 161677), '2023/12/13 11:35:32.007689', '[-58.0]']
[datetime.datetime(2023, 12, 13, 12, 37, 19, 161677), '2023/12/13 11:35:32.015377', '[-62.0]']
[datetime.datetime(2023, 12, 13, 12, 37, 19, 161677), '2023/12/13 11:35:32.023066', '[-53.0]']
[datetime.datetime(2023, 12, 13, 12, 37, 19, 161677), '2023/12/13 11:35:32.030754', '[-31.0]']
[datetime.datetime(2023, 12, 13, 12, 37, 19, 161677), '2023/12/13 11:35:32.038443', '[-33.0]']
